#Step by step design for the two resonator single xmon design. 


## Sections
### I. Transmon only
1. Prepare the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>
1. (Extra: Calculate surface EPR.) <br>

### II. Resonator only
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Calculate EPR of substrate. <br>
    
### III. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### IV. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### V. Analyze the coupled  transmon 2 resonator system. 
1. Finite Element Eigenmode Analysis
1. Identify the mode you want. The mode can inclusively be from 1 to setup.n_modes.
1. Set variables in the Ansys design. As before, we seek 2 modes.
1. Set up the simulation and specify the variables for the sweep.
1. Plot the E-field on the chip's surface.
1. Specify the junctions in the model; in this case there are 2 junctions.
1. Find the electric and magnetic energy stored in the substrate and the system as a whole.
1. Perform EPR analysis for all modes and variations.

In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
%metal_heading Qiskit Metal works fine! - Here are the design components!

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL

from qiskit_metal.qlibrary.qubits.transmon_pocket_cl_fl import TransmonPocketCL_FL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.tunable_coupler_01 import TunableCoupler01

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

#from qiskit_metal.qlibrary.terminations.launchpad_wb_simplecoupler import LaunchpadWirebondSimple
#from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
import numpy as np
from collections import OrderedDict

from qiskit_metal.qlibrary.qubits.transmon_cross_ebru_temp import TransmonCross_ebru_temp

from qiskit_metal.qlibrary.qubits.transmon_cross_fl_ebru_temp import TransmonCrossFL_ebru_temp
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl_ebru_temp import TransmonCrossFL_CL_ebru_temp


In [4]:
design = metal.designs.DesignPlanar()

design.chips.main.size['size_x'] = '5mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['size_z'] = '-725um'
gui = MetalGUI(design)

# Resonator length estimate

In [5]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"


find_resonator_length(6.0, 10, 6, 4)

'5.071952814490852 mm'

# Xmon qubits 

In [95]:
design.overwrite_enabled = True
options =  dict(
    connection_pads=dict(
        readout = dict(loc_W=+1, loc_H=+1),
        storage = dict(loc_W=-1, loc_H=-1, )
    ))
    
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl_fl import TransmonPocketCL_FL


# Q1 = TransmonCrossFL_CL_ebru_temp(design, 'Q1', options = dict(make_extra_rect=False, pos_x = '0.450mm', pos_y='1.410mm', 
#                                                      cross_length='40um', cross_width='10um', cross_gap='15um',
#                                                     # cross_gap='3um',
#                                                  connection_pads = dict(
#                                                      readout = dict(connector_location = '90', lead_gap='6um',
#                                                      claw_length='30um', claw_width='5um', ground_spacing='2um', 
#                                                                     claw_gap='5um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
#                                                  fl_options = dict(t_shift='3.5um' , t_offset='-6um', t_inductive_gap='4.5um', 
#                                                                    t_top='10.5um', t_width='5um', t_gap='3um'),
#                                                  cl_options= dict(c_offset='-40um', c_ground_gap='38um', c_width='5um', 
#                                                                   c_gap='3um', translate_x='0um', c_angle='90'),
#                                                  ))


Q1 = TransmonPocketCL_FL(design, 'Q1', options = dict( pos_x = '0.65mm', pos_y='1.410mm', orientation='90',
                                                            connection_pads=dict(
                                    a = dict(loc_W='120um', loc_H='-100um', pad_width='50um', cpw_extend = '50um'),
                                    
                                    #b = dict(loc_W=-1, loc_H=-1, pad_width='125um', cpw_extend = '50um', pad_height='60um'),
                                    #c = dict(loc_W=+1, loc_H=+1, pad_width='110um', cpw_extend = '50um')
    ),
    pad_width='100um', pad_gap='140um', 
    pocket_width='450um', pocket_height='550um', 
    cl_pocket_edge = '90',
    cl_off_center = '2.5um',
    cl_gap='3um', cl_width='5um',
                                                      
                                                 ))


Q2 = TransmonCrossFL_CL_ebru_temp(design, 'Q2', options = dict(make_extra_rect=False, pos_x = '0.2mm', pos_y='1.410mm', orientation=270, 
                                                     cross_length='160um', cross_width='36um', cross_gap='30um',
                                                    
                                                 connection_pads = dict(
                                                     readout = dict(connector_location = '0', lead_gap='6um',
                                                     claw_length='140um', claw_width='10um', ground_spacing='3um', 
                                                                    claw_gap='8um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
                                                 fl_options = dict(t_shift='-200um' , t_offset='-200um', t_inductive_gap='4.5um', 
                                                                   t_top='10.5um', t_width='5um', t_gap='3um'),
                                                 cl_options= dict(c_offset='30um', c_ground_gap='38um', c_width='5um', 
                                                                  c_gap='3um', translate_x='-20um', translate_y='-50um',  c_angle='0'),
                                                 rect_options= dict(rect_width='40um', rect_length='8um')))

gui.rebuild()
gui.autoscale()

# Flux Lines and Flux Launchpads

In [96]:

import numpy as np
from collections import OrderedDict


# anchors1f = OrderedDict()
# anchors1f[0] = np.array([0.18, 1.12])
# anchors1f[1]=np.array([-1.15, 0.5])


launch_flux_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                           pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um', 
                           pos_x='-1.5mm', pos_y='1.412mm', orientation='0', lead_length='30um')
lf = LaunchpadWirebond(design, 'Launch_Flux', options = launch_flux_options)


flux_line_options = Dict(
        #lead = dict(start_straight='100um', end_straight='100um'),
        fillet='30um', 
        trace_width = '5um', trace_gap = '3um', 
        pin_inputs=Dict(
            start_pin=Dict(component='Q2', pin='flux_line'),
            end_pin=Dict(componentl=lf.name, pin='tie')), )
flux_line_Q2 = RouteAnchors(design,'Flux_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Flux',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 #end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '20um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                             #anchors = anchors1f
                                            ))

gui.rebuild()

01:15AM 31s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 31s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 31s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 31s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [97]:



# anchors2fl = OrderedDict()
anchors2f = OrderedDict()
anchors2f[0] = np.array([1.2, 1.25])
anchors2f[1]=np.array([1.2, 0.5])


launch_flux2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5', pos_y='0.5mm', orientation='180', lead_length='30um')
lf2 = LaunchpadWirebond(design, 'Launch_Flux2', options = launch_flux2_options)




flux_line_Q1 = RoutePathfinder(design,'Flux_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Flux2',
                                                    pin='tie')),
                                            lead=Dict(
                                                 #start_straight='175um',
                                                 #end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '20um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors=anchors2f,
                                            ))


gui.rebuild()

01:15AM 35s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 35s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [ ]:
print("RUn")

# Charge Lines and Charge Launchpads

In [98]:






launch_charge_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5mm', pos_y='1.405mm', orientation='180', lead_length='30um')

lc = LaunchpadWirebond(design, 'Launch_Charge', options = launch_charge_options)








#anchors1ch = OrderedDict()
#anchors1ch[0] = np.array([1.78, 0.114])


charge_line_Q1 = RoutePathfinder(design,'Charge_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='Charge_Line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='175um',
                                                end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors1ch
                                            ))

gui.rebuild()

01:15AM 39s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 39s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [ ]:

launch_charge2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.5mm', pos_y='0.5mm', orientation='0', lead_length='30um')

lc2 = LaunchpadWirebond(design, 'Launch_Charge2', options = launch_charge2_options)
anchors2ch = OrderedDict()
anchors2ch[0] = np.array([-1.75,2.0])
charge_line_Q2 = RoutePathfinder(design,'Charge_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge2',
                                                    pin='tie')),
                                            lead=Dict(
#                                                 start_straight='175um',
                                                 end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors2ch
                                            ))

gui.rebuild()


# Readout Tee's and Readout Resonators

In [99]:

TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='-0.85mm',
                                             pos_y='2.6mm',
                                             coupling_length='460um', coupling_space='3um', orientation=0, 
                                                 open_termination=False))





#3.58

options2 = Dict(
     total_length='4.250mm',
     hfss_wire_bonds = True,
     fillet='40um',
     lead = dict(start_straight='250um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='TQ2',
             pin='second_end'),
         start_pin=Dict(
             component='Q2',
             pin='readout')),
     meander=Dict(spacing='200um',
             asymmetry='0um'))
read_q2 = RouteMeander(design, 'read_q2', options=options2)



gui.rebuild()
gui.autoscale()


01:15AM 43s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 43s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [100]:

TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='1.07mm',
                                             pos_y='2.6mm',
                                              coupling_length='400um', coupling_space='4um', open_termination=False, orientation='0'))

options1 = Dict(
    total_length='3.9mm',
    hfss_wire_bonds = True,
    fillet='40um',
    lead = dict(start_straight='50um'),
    pin_inputs=Dict(
        start_pin=Dict(
            component='TQ1',
            pin='second_end'),
        end_pin=Dict(
            component='Q1',
            pin='a')))
read_q1 = RouteMeander(design, 'read_q1', options=options1)
gui.rebuild()
gui.autoscale()

01:15AM 46s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 46s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


# Feedline and Feedline Launchpads 

In [101]:
launch_feed_left_options = dict(trace_width= '10um',pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.5mm', pos_y='2.6mm', orientation='0', lead_length='30um')
l_feed_left = LaunchpadWirebond(design, 'Launch_Feed_Left', options = launch_feed_left_options)


launch_feed_right_options = dict(trace_width= '10um', pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5mm', pos_y='2.6mm', lead_length='30um', orientation='180')
l_feed_right = LaunchpadWirebond(design, 'Launch_Feed_Right', options = launch_feed_right_options)




# anchors1c = OrderedDict()
# anchors1c[0] = np.array([-0.65, 0.56079])



ops_oF = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ2', 'pin': 'prime_start'}, 
             'end_pin': {'component': 'Launch_Feed_Left', 'pin': 'tie'}},
            #'fillet':'80um'
            }
cpw_openFirst = RouteStraight(design, 'cpw_openFirst', options=ops_oF)




ops_oM = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ1', 'pin': 'prime_start'}, 
             'end_pin': {'component': 'TQ2', 'pin': 'prime_end'}},
            #'anchors': anchors1c,
            #'fillet':'50um'
            }
cpw_openMid = RouteStraight(design, 'cpw_openMid', options=ops_oM)





# ops_oLast = {'hfss_wire_bonds': 'True',
#             'pin_inputs': 
#             {'start_pin': {'component': 'TQ1', 'pin': 'prime_end'}, 
#              'end_pin': {'component': l_feed_right.name, 'pin': 'tie'}},
#             'fillet':'80um'
#             }
# cpw_openLast = RouteStraight(design, 'cpw_openLast', options=ops_oLast)



gui.rebuild()
gui.autoscale()

01:15AM 50s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:15AM 50s WARNING [check_lengths]: For path table, component=Flux_Line_2, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [ ]:
%metal_heading Now analyzing the design: HFSS & EPR

# Running for an individual XMon

In [79]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

eig_qb = EPRanalysis(design, "hfss")

eig_qb.sim.setup.vars.Lj = '10 nH'
eig_qb.sim.setup.vars = Dict({'Lj': '10 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

In [80]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb.sim.setup

In [85]:
eig_qb.sim.setup.max_passes = 16
eig_qb.sim.setup.vars.Lj = '10 nH'
eig_qb.sim.setup.vars = Dict({'Lj': '10 nH', 'Cj': '3 fF'})

eig_qb.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 4)
eig_qb.sim.setup.min_passes = 10
eig_qb.sim.setup.n_modes = 1

gui.rebuild()
gui.autoscale()



In [88]:
eig_qb.sim.renderer._options['max_mesh_length_jj'] = '3um'
eig_qb.sim.renderer._options['max_mesh_length_port'] = '3um'

In [91]:

design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



eig_qb.sim.run(name="Xmon_only_new", components=['Q2'], open_terminations=[], box_plus_buffer = True)
eig_qb.sim.plot_convergences()

INFO 09:02PM [connect_design]: 	Opened active design
	Design:    Xmon_only_new_hfss [Solution type: Eigenmode]
INFO 09:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:02PM [analyze]: Analyzing setup Setup
09:05PM 58s INFO [get_f_convergence]: Saved convergences to C:\Users\WANGLAB\Documents\GitHub\qiskit-metal\tutorials\4 Analysis\A. Core - EM and quantization\hfss_eig_f_convergence.csv


The last variables you pass to the `run()` or `sim.run()` methods, will be stored in the `sim.setup` dictionary under the key `run`. You can recall the information passed by either accessing the dictionary directly, or by using the print handle below.

In [ ]:
eig_qb.sim.plot_convergences()

In [ ]:
# eig_qb.setup.run    <- direct access
eig_qb.sim.print_run_args()

In [93]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'
eig_qb.run_epr()

Design "Xmon_only_new_hfss" info:
	# eigenmodes    1
	# variations    1
Design "Xmon_only_new_hfss" info:
	# eigenmodes    1
	# variations    1


 C:\Users\WANGLAB\anaconda3\envs\qiskit-metal\Lib\site-packages\pyEPR\project_info.py: 239



        energy_elec_all       = 2.42277039473135e-25
        energy_elec_substrate = 2.23332259257517e-25
        EPR of substrate = 92.2%

        energy_mag    = 2.09690701355333e-27
        energy_mag % of energy_elec_all  = 0.9%
        

Variation 0  [1/1]


 C:\Users\WANGLAB\anaconda3\envs\qiskit-metal\Lib\site-packages\pyEPR\core_distributed_analysis.py: 1101
 C:\Users\WANGLAB\anaconda3\envs\qiskit-metal\Lib\site-packages\pyEPR\core_distributed_analysis.py: 1102
 C:\Users\WANGLAB\anaconda3\envs\qiskit-metal\Lib\site-packages\pyEPR\core_distributed_analysis.py: 1245



  Mode 0 at 5.14 GHz   [1/1]
    Calculating ℰ_magnetic,ℰ_electric


...

###### Numerical diagonalization

Lj,10
0,4932.82


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

,,0
Lj,,
10,0,216.54


# Capacitance matrix 

In [94]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

# example: update single setting
c1.sim.setup.max_passes = 10
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

c1.sim.run(components=['Q1', 'Q2'], open_terminations=[])
c1.sim.capacitance_matrix

INFO 09:39PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:39PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:39PM [analyze]: Analyzing setup Setup
INFO 09:40PM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmp4xdy4xwq.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 09:40PM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmp1j61nqna.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 09:40PM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmp8egolq2z.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 09:40PM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmp1kua0geb.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 

,cross_Q2,ground_main_plane,pad_bot_Q1,pad_top_Q1
cross_Q2,87.42823,-84.12185,-0.35855,-1.46081
ground_main_plane,-84.12185,193.96004,-23.22842,-22.22070
pad_bot_Q1,-0.35855,-23.22842,27.33465,-2.41914
pad_top_Q1,-1.46081,-22.22070,-2.41914,27.31718


ValueError: Capacitance matrix is not the right size

In [76]:

from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem

opt1 = dict(
    node_rename = {'coupling_pad_Q1': 'coupling'}, 
    cap_mat = c1.sim.capacitance_matrix,
    ind_dict = {('pad_bot_Q1', 'pad_top_Q1'): 1},  # placeholder inductance here; only used for node-basis transformation and reduction
    jj_dict = {('pad_bot_Q1', 'pad_top_Q1'):'j1'},
)
cell_1 = Cell(opt1)




In [77]:
fluxonium = Subsystem(name='fluxonium', sys_type='FLUXONIUM', nodes=['j1'], q_opts={'EJ':5570, 'EL':640, 'flux': .5})
composite_sys = CompositeSystem(
    subsystems=[fluxonium], 
    cells=[cell_1], 
    grd_node='ground_main_plane')

cg = composite_sys.circuitGraph()

In [78]:
hilbertspace = composite_sys.create_hilbertspace()
hilbertspace = composite_sys.add_interaction()
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

Fluxonium-----------| [Fluxonium_22]
                    | EJ: 5570
                    | EC: 1300.1785677075857
                    | EL: 640
                    | flux: 0.5
                    | cutoff: 110
                    | truncated_dim: 10
                    |
                    | dim: 110




In [26]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

Bare spectra:   0%|          | 0/101 [00:00<?, ?it/s]

Dressed spectrum:   0%|          | 0/101 [00:00<?, ?it/s]

ValueError: Number of requested eigen vals/vecs must be <= N.

In [ ]:
eig_qb.sim.plot_fields('main')  

hfss = eig_qb.sim.renderer
hfss.clear_fields(['Mag_E1'])
hfss.set_mode(1, "Setup")
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

In [ ]:
eig_qb.sim.clear_fields()

# Xmon and resonators together

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [ ]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'
gui.rebuild()
gui.autoscale()

In [ ]:
eig_rd = EPRanalysis(design, "hfss")

eig_rd.sim.setup.vars.Lj = '550 nH'
eig_rd.sim.setup.vars = Dict({'Lj': '550 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

# Qubit 1 related simulations

In [ ]:
eig_rd.sim.setup.n_modes =1

# eig_rd.sim.setup.min_passes = 1
# eig_rd.sim.setup.max_passes = 1

eig_rd.sim.setup.min_passes = 15
eig_rd.sim.setup.max_passes = 20


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'


#eig_rd.sim.run(name="xmon_and_readout_storage_final", components=['Q1', 'read_q1', 'Store_Q1_1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_and_readout_87", components=['Q1', 'read_q1', 'TQ1'], open_terminations=[('TQ1', 'prime_start'), ('TQ1', 'prime_end')], box_plus_buffer = False)


eig_rd.sim.run(name="readout_transmon_hfss33", components=['read_q1', 'TQ1', 'Q1'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = True)
#eig_rd.sim.run(name="storage_and_xmon_and_readout_final3", components=['Q2', 'Store_Q2_2', 'read_q2', 'TQ2', 'TQ1', 'Launch_Feed_Left', 'Launch_Feed_Right', 'cpw_openFirst', 'cpw_openMid', 'cpw_openLast'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_chargeline", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1','end')])
# eig_rd.sim.run(name="open_24_131", components=['Q1', 'Store_Q1_1', 'Launch_Storage'], open_terminations=[('Launch_Storage','tie')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_coupling_PART96", components=['Q1','Store_Q1_1', 'Storage_Line_1', 'otg11', 'stg11'], open_terminations=[('Storage_Line_1', 'end')], box_plus_buffer = False)
#eig_rd.sim.run(name="ro_111622", components=['Q2', 'read_q2', 'TQ2', 'Store_Q2_2'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="full_set2", components=['Q2', 'Store_Q2_2' 'read_q2', 'TQ2'], open_terminations=[], box_plus_buffer = False)

#eig_rd.sim.run(name="xmon_and_ro_set1", components=['Q1',  'read_q1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_loss_fluxpad2", components=['Q2', 'Store_Q2_2', 'Launch_Flux2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'
eig_rd.run_epr()

# Qubit 2 related simulations

In [ ]:
eig_rd.sim.setup.n_modes =1

eig_rd.sim.setup.min_passes = 1
eig_rd.sim.setup.max_passes = 1

# eig_rd.sim.setup.min_passes = 11
# eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()






#eig_rd.sim.run(name="storage_coupling_PART92", components=['Q2','Store_Q2_2', 'Storage_Line_2', 'otg11_2', 'stg11_2'], open_terminations=[('Storage_Line_2', 'end')], box_plus_buffer = False)




#eig_rd.sim.run(name="xmon_and_readout_storage_ayoy_39", components=['Q2', 'read_q2', 'Store_Q2_2', 'TQ2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.run(name="xmon_and_readout_PART90", components=['Q2', 'read_q2', 'TQ2'], open_terminations=[('TQ2', 'prime_start'), ('TQ2', 'prime_end')], box_plus_buffer = False)


#eig_rd.sim.run(name="readout4", components=['read_q1', 'TQ1', 'Q1'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_and_xmon_and_readout", components=['Q1', 'Store_Q1', 'read_q1', 'TQ1', 'Launch_Feed_Left', 'Launch_Feed_Right', 'cpw_openRight', 'cpw_openLeft'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_chargeline", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1','end')])
# eig_rd.sim.run(name="open_24_131", components=['Q1', 'Store_Q1_1', 'Launch_Storage'], open_terminations=[('Launch_Storage','tie')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_coupling_ayoy", components=['Q1', 'Store_Q1_1', 'stg11', 'Storage_Line_1', 'otg11'], open_terminations=[('otg11', 'open'), ('Storage_Line_1', 'end')], box_plus_buffer = False)
#eig_rd.sim.run(name="ro_111622", components=['Q2', 'read_q2', 'TQ2', 'Store_Q2_2'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="full_set2", components=['Q2', 'Store_Q2_2' 'read_q2', 'TQ2'], open_terminations=[], box_plus_buffer = False)

#eig_rd.sim.run(name="xmon_and_ro_set1", components=['Q1', 'Store_Q1_1', 'read_q1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_loss_fluxpad2", components=['Q2', 'Store_Q2_2', 'Launch_Flux2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.sim.setup.n_modes =3

eig_rd.sim.setup.min_passes = 1
eig_rd.sim.setup.max_passes = 1

# eig_rd.sim.setup.min_passes = 15
# eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()


eig_rd.sim.run(name="chargecoupling", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1', 'end')], box_plus_buffer = False)


eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.sim.setup

Recover eigenmode frequencies for each variation.

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:

hfss = eig_rd.sim.renderer
hfss.clear_fields(['Mag_E1'])
hfss.set_mode(1, "Setup")
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

### EPR Analysis
Find the electric and magnetic energy stored in the readout system.

In [ ]:

eig_rd.sim.setup.name

In [ ]:
# eig_rd.setup
# eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
# eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

# eig_rd.setup

eig_rd.setup
eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'

eig_rd.setup

In [ ]:

eig_rd.run_epr()

In [ ]:
eig_rd.sim.setup.vars.Lj 

In [ ]:
eig_rd.sim.close()

# LOM

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

# example: update single setting
c1.sim.setup.max_passes = 10
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup


c1.sim.run(components=['Q1', 'Store_Q1_1','Storage_Line_1', 'Launch_Storage'], open_terminations=[('Storage_Line_1', 'start')])


In [ ]:
c1.sim.capacitance_matrix